In [256]:
import pandas as pd

# Load data

Data is loaded into two separate DataFrames, one for the electricity data and one for the weather data

In [257]:
df_el = pd.read_csv("./data/ewz_stromabgabe_netzebenen_stadt_zuerich.csv")

In [258]:
frames = []
for i in range(15, 25, 1):
    frames.append(pd.read_csv(f"./data/ugz_ogd_meteo_h1_20{i}.csv"))
df_wthr = pd.concat(frames)

In [259]:
df_hlds = pd.read_csv("./data/zh_feiertage.csv")

In [260]:
df_vctn = pd.read_csv("./data/zh_schulferien.csv")

# Format data

The underlying data is not yet in a usable format for this project, therefore before any data analysis can be done, first the data has be put into a usable format

## Formatting Electricity Data

The following steps have to be performed to put the electricity data into the desired format:

- Convert the "Timestamp" column into a "DateTime" column
- Set the newly created "Date" column as the index -> timeseries data
- Drop the now unused "Timestamp" column
- Sum up all quarter hour datapoints up to receive columns with one hour accuracy, since the weather data     resolution is also only hourly

In [261]:
df_el["Date"] = pd.to_datetime(df_el["Timestamp"], utc=True)
df_el.set_index(["Date"], inplace=True)
df_el.drop(["Timestamp"], inplace=True, axis=1)
df_el = df_el.resample("h").sum()
df_el.reset_index(inplace=True)

## Formatting Weather Data

The following steps have to be performed to put the weather data into the desired format:

- Convert the "Datum" column into a "DateTime" column
- Set the newly created "Date" column as the index -> timeseries data
- Only keep entries for the measurement station "Zch_Stampfenbachstrasse" -> Station with most Datapoints
- Extend the values in the "Parameter" column with the values in the "Einheit" column
- Drop the unused columns "Datum", "Intervall", "Standort" and "Einheit"
- Pivot the table to have all Parameters as columns with their respective "Wert" as values. In this step the DataFrame is also reduced to one entry per hour from the previous eight entries


In [262]:
df_wthr["Date"] = pd.to_datetime(df_wthr["Datum"], utc=True)
df_wthr.set_index(["Date"], inplace=True)
df_wthr = df_wthr[df_wthr["Standort"] == "Zch_Stampfenbachstrasse"]
df_wthr['Parameter'] = df_wthr['Parameter'] + ' [' + df_wthr['Einheit'] + "]"
df_wthr.drop(["Datum", "Intervall", "Standort", "Einheit"], inplace=True, axis=1)
df_wthr = df_wthr.pivot(columns='Parameter', values='Wert')
df_wthr.reset_index(inplace=True)

## Formatting Holiday Data

The following steps have to be performed to put the holiday data into the desired format:

- Convert the "Datum" column into a "DateTime" column
- Set the newly created "Date" column as the index -> timeseries data
- Drop the unused columns "Datum" and "Feiertag"
- Rename the remaining column "Typ" to "Holiday Type"
- Expand the DataFrame so each hour of each day has an entry with the value of the respective holiday type
- Reset the index so the "Date" entry is once again a column


In [263]:
df_hlds["Date"] = pd.to_datetime(df_hlds["Datum"], utc=True)
df_hlds["Date"] = df_hlds["Date"] - pd.Timedelta(hours=1)
df_hlds.set_index(["Date"], inplace=True)
df_hlds.drop(["Datum", "Feiertag"], inplace=True, axis=1)
df_hlds.rename(columns={"Typ": "Holiday Type"}, inplace=True)

In [264]:
expanded_rows = []
for idx, row in df_hlds.iterrows():
    date_range = pd.date_range(start=idx, end=idx + pd.Timedelta(hours=23), freq="h")
    expanded_rows.append(pd.DataFrame({'Holiday': 1}, index=date_range))

df_hlds = pd.concat(expanded_rows)

In [265]:
df_hlds.reset_index(inplace=True)
df_hlds.rename(columns={"index": "Date"}, inplace=True)

## Formatting School Vacation Data

The following steps have to be performed to put the school vacation data into the desired format:

- Convert the "Datum" column into a "DateTime" column
- Set the newly created "Date" column as the index -> timeseries data
- Drop the unused columns "Datum" and "Ferien"
- Expand the DataFrame so each hour of each day has an entry with value 1 for school vacation from a respective "Start" to "Stop"
- Reset the index so the "Date" entry is once again a column


In [266]:
df_vctn["Date"] = pd.to_datetime(df_vctn["Datum"], utc=True)
df_vctn["Date"] = df_vctn["Date"] - pd.Timedelta(hours=1)
df_vctn.set_index(["Date"], inplace=True)
df_vctn.drop(["Datum", "Ferien"], inplace=True, axis=1)

In [267]:
expanded_rows = []
start_date = None
for idx, row in df_vctn.iterrows():
    if row['Start/Stop'] == 'Start':
        start_date = idx  # Store the Start date
    elif row['Start/Stop'] == 'Stop' and start_date is not None:
        date_range = pd.date_range(start=start_date, end=idx - pd.Timedelta(hours=1), freq="h")
        expanded_rows.append(pd.DataFrame({'Vacation': 1}, index=date_range))
        start_date = None

df_vctn = pd.concat(expanded_rows)


In [268]:
df_vctn.reset_index(inplace=True)
df_vctn.rename(columns={"index": "Date"}, inplace=True)

## Combine Tables

From the previously formatted DataFrames, create a single one containing all the information

Afterwards, fill all "NaN" values for the columnns "Vacation" and "Holiday_Type", because if they are "NaN" this means that these days are not vacation days or holidays.

In the end, drop all rows where no electricity data is available, since they are not usable for this project.

In [269]:
df = pd.merge(df_el, df_wthr, how='outer', on="Date")
df = pd.merge(df, df_vctn, how="left", on="Date")
df = pd.merge(df, df_hlds, how="left", on="Date")

In [270]:
df["Vacation"] = df["Vacation"].fillna(0)
df["Holiday"] = df["Holiday"].fillna(0)

# Save Data

The formatted data is once again stored in an external file for quick an easy reload in other notebooks

In [271]:
df.to_parquet("./data/formatted_source_data.parquet", engine="pyarrow")